In [25]:
import pandas as pd
from client import BitcoinClient
from address_type import address_type
import pickle

In [2]:
change_df = pd.read_csv('../transaction-extractor/output.csv', low_memory=False)

In [3]:
change_df.head()

,Transaction ID,Change Address Index
0,fc12dfcb4723715a456c6984e298e00c479706067da81b...,1.0
1,2c58bcf09b1eabc1aff81bb7ad8efe0368cb9db7b95446...,1.0
2,533a4eb47a1ad0c4f4357a16f1e1f6cc01dbb25c95ed61...,0.0
3,60f55e0a7edff5b9f00f3b1dd655e3b47f2ded0b1522f7...,1.0
4,2f5aa31095d58feaf2f8a5b820508763303e7c79f30c61...,1.0


In [4]:
change_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5382 entries, 0 to 5381
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Transaction ID        5382 non-null   object 
 1   Change Address Index  5382 non-null   float64
dtypes: float64(1), object(1)
memory usage: 84.2+ KB


In [5]:
client = BitcoinClient()
transaction_dict = {}
for txid in change_df['Transaction ID']:
    transaction = client.transaction_by_id(txid)
    transaction_dict[txid] = transaction

In [6]:
transaction_dict['00116413937ea7fafde16076091676e9c2e0d4aee8a36700574ce082b9442e53']

{'txid': '00116413937ea7fafde16076091676e9c2e0d4aee8a36700574ce082b9442e53',
 'hash': '00116413937ea7fafde16076091676e9c2e0d4aee8a36700574ce082b9442e53',
 'version': 1,
 'size': 258,
 'vsize': 258,
 'weight': 1032,
 'locktime': 0,
 'vin': [{'txid': 'b716f6f92d6a561a28ca352f5a70f24c3121e3516d6cdf44af3d905c931483ea',
   'vout': 1,
   'scriptSig': {'asm': '30450221008cafe17eacb90d9bf273495444555ff78c93284f17428a096b23ed481318a5e8022075de40715efdcfa8bb50ebb6e1856853c52b69dc30337f15ee26d386032f72e4[ALL] 0440e391806e70aa7ee55b21a67c02c24e1d9dce3af79153ccd14d913c57e70965d8e48af85d6f0c5ae1df6c0f427b5ee7ecb22213cd7091036a95ba6bcde517e8',
    'hex': '4830450221008cafe17eacb90d9bf273495444555ff78c93284f17428a096b23ed481318a5e8022075de40715efdcfa8bb50ebb6e1856853c52b69dc30337f15ee26d386032f72e401410440e391806e70aa7ee55b21a67c02c24e1d9dce3af79153ccd14d913c57e70965d8e48af85d6f0c5ae1df6c0f427b5ee7ecb22213cd7091036a95ba6bcde517e8'},
   'sequence': 4294967295}],
 'vout': [{'value': Decimal('0.00008000'

In [16]:
column_list = {
    "output": pd.Series(dtype="bool"),
    "txid": pd.Series(dtype="str"),
    "version": pd.Series(dtype="int"),
    "size": pd.Series(dtype="int"),
    "vsize": pd.Series(dtype="int"),
    "weight": pd.Series(dtype="int"),
    "locktime": pd.Series(dtype="int"),
    "index": pd.Series(dtype="int"),
    "value": pd.Series(dtype="float"),
    "address_type": pd.Series(dtype="int"),
    "is_change": pd.Series(dtype="int"),
}

In [17]:
def is_change(txid, index):
    try:
        row = change_df[change_df['Transaction ID'] == txid]
        return int(row['Change Address Index']) == index
    except:
        return False

In [18]:
df_list = []

In [22]:
del client
client = BitcoinClient()
for txid, transaction in transaction_dict.items():
    try:
        temp_df = pd.DataFrame(columns=column_list)
        coinbase = False
        
        for vout in transaction['vout']:
            new_row = {
                'output': True,
                'txid': txid,
                'version': transaction['version'],
                'size': transaction['size'],
                'vsize': transaction['vsize'],
                'weight': transaction['weight'],
                'locktime': transaction['locktime'],
                'index': vout['n'],
                'value': float(vout['value']),
                'address_type': address_type(vout["scriptPubKey"]["address"]),
                'is_change': is_change(txid, vout['n'])
            }
            temp_df.loc[len(temp_df.index)] = new_row
        
        for vin in transaction['vin']:
            if 'coinbase' in vin:
                coinbase = True
                continue
            input_transaction = client.transaction_by_id(vin['txid'])
            new_row = {
                'output': False,
                'txid': vin['txid'],
                'version': input_transaction['version'],
                'size': input_transaction['size'],
                'vsize': input_transaction['vsize'],
                'weight': input_transaction['weight'],
                'locktime': input_transaction['locktime'],
                'index': vin['vout'],
                'value': float(input_transaction['vout'][vin['vout']]['value']),
                'address_type': address_type(input_transaction['vout'][vin['vout']]["scriptPubKey"]["address"]),
                'is_change': False
            }
            temp_df.loc[len(temp_df.index)] = new_row
            
        if coinbase:
            continue
        df_list.append(temp_df)
    except:
        continue

C:\Users\futaba\AppData\Local\Temp\ipykernel_36668\3086613383.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(row['Change Address Index']) == index
C:\Users\futaba\AppData\Local\Temp\ipykernel_36668\3086613383.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(row['Change Address Index']) == index
C:\Users\futaba\AppData\Local\Temp\ipykernel_36668\3086613383.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(row['Change Address Index']) == index
C:\Users\futaba\AppData\Local\Temp\ipykernel_36668\3086613383.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(row['C

In [20]:
df_list[0]

,output,txid,version,size,vsize,weight,locktime,index,value,address_type,is_change
0,True,fc12dfcb4723715a456c6984e298e00c479706067da81b...,1,520,520,2080,0,0,2.000000,1,False
1,True,fc12dfcb4723715a456c6984e298e00c479706067da81b...,1,520,520,2080,0,1,4.697496,1,True
2,False,97d7279d666fe5bd48799c33a7197e92a7dc281169008e...,1,225,225,900,0,1,0.271166,1,False
3,False,9c37723164c3ea6e2a79a4304af15afb7e2a6c8a6a717d...,1,4208,4208,16832,0,0,1.538040,1,False
4,False,a19180958b766780e6bc9e97579f02bc71433d10da3197...,1,225,225,900,0,0,4.896290,1,False


In [24]:
len(df_list)

5318

In [27]:
with open('transactions.df.list.pkl', 'wb') as f:
    pickle.dump(df_list, f, pickle.HIGHEST_PROTOCOL)